# ABOR example

This notebook will first setup all the supporting data (i.e portfolios, instruments, transactions, quotes). We will then do the following:

1) Create a chart of accounts
2) Add charts to chart of accounts
3) Create posting rules
4) Create ABOR configuration
5) Create ABOR
6) Create journal entry lines
7) Create GL profile mappings
8) Create GL profile
9) Create trial balance
10) Run trial balance check


## Setup 
First we will create the valuation recipe and transaction types which are required for the commands.

In [35]:
# Import general purpose packages
import json
import os
import time
import pandas as pd
import logging
import argparse

# Import LUSID specific packages
from lusid.utilities import ApiClientFactory as LusidApiClientFactory
import lumipy
import lusid
import lusid.api as la
import lusid.models as models
import lusid_drive
from lusid_workflow.rest import ApiException
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidjam import RefreshingToken

# Create loggers
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# Authenticate our user and create API client
secrets_path = os.getenv("FBN_SECRETS_PATH")
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook"
)

# Get URL and use it to configure Drive SDKs
configuration_recipe_api = api_factory.build(la.ConfigurationRecipeApi)
api_url = api_factory.api_client.configuration._base_path.replace("api","")
drive_configuration = lusid_drive.Configuration(host=api_url + "drive")
drive_configuration.access_token = api_factory.api_client.configuration.access_token
drive_configuration = lusid_drive.Configuration(
    host=f"{configuration_recipe_api.api_client.configuration.host[:-4]}/drive"
)
drive_configuration.access_token = (
    configuration_recipe_api.api_client.configuration.access_token
)
with lusid_drive.ApiClient(drive_configuration) as api_client:
    files_api = lusid_drive.FilesApi(api_client)
    folders_api = lusid_drive.FoldersApi(api_client)
# Setup Drive, and Lumipy Clients
drive_client = lusid_drive.ApiClient(drive_configuration)

def run_luminesce(luminesce_string):  
    client = lumipy.get_client(api_secrets_filename=secrets_path, token=RefreshingToken())
    df = client.query_and_fetch(luminesce_string)
    return df

In [124]:
# Attempt to create the folder structure in LUSID Drive
# All files will live in a 'workflow-demo-data' parent folder and then 'trades', 'prices' and 'source-data' subfolders.
drive_parent_folder='abor-demo-data'
drive_price_subfolder = "prices"
local_filepath = "./data/"


def create_folder_drive(path, name):
    try:
        response = folders_api.create_folder(
            lusid_drive.models.CreateFolder(path=path, name=name)
        )
        print(f"Folder '{path}{name}' created successfully!")
    except lusid_drive.rest.ApiException as e:
        if json.loads(e.body)["code"] == 664:
            print(f"Folder '{path}{name}' already exists in Drive.")
        else:
            raise


create_folder_drive("/", drive_parent_folder)
create_folder_drive(f"/{drive_parent_folder}/", drive_price_subfolder)
create_folder_drive(f"/{drive_parent_folder}/{drive_price_subfolder}", "processed")


# Define function that will upload our files into Drive from our local data subfolder
def upload_files_drive(subfolder, filename):
    try:
        with open(local_filepath + filename, "rb") as data:
            response = files_api.create_file(
                x_lusid_drive_filename=filename,
                x_lusid_drive_path=drive_parent_folder + "/" + subfolder,
                content_length=os.stat(local_filepath + filename).st_size,
                body=data.read(),
            )
            print(f"File '/{drive_parent_folder}/{subfolder}/{filename}' uploaded successfully!")
    except lusid_drive.rest.ApiException as e:
        if json.loads(e.body)["code"] == 671:
            print(f"File '/{drive_parent_folder}/{subfolder}/{filename}' already exists in Drive.")
        else:
            raise


file_details = {
    "aborquotes.xlsx": drive_price_subfolder,
    "daily_quotes.xlsx": drive_price_subfolder,
}

for k, v in file_details.items():
    upload_files_drive(subfolder=v, filename=k)

Folder '/abor-demo-data' already exists in Drive.
Folder '/abor-demo-data/prices' already exists in Drive.
Folder '/abor-demo-data/pricesprocessed' already exists in Drive.
File '/abor-demo-data/prices/aborquotes.xlsx' uploaded successfully!
File '/abor-demo-data/prices/daily_quotes.xlsx' already exists in Drive.


In [113]:
def create_recipe(api_factory):
    recipes_api = api_factory.build(lusid.api.ConfigurationRecipeApi)

    scope = "Finbourne-Examples"
    recipe_code = "marketValue"

    configuration_recipe = models.ConfigurationRecipe(
        scope=scope,
        code=recipe_code,
        market=models.MarketContext(
            market_rules=[
                models.MarketDataKeyRule(
                    key="Quote.ClientInternal.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Price",
                    field="mid",
                    quote_interval="100D.0D",
                ),
                models.MarketDataKeyRule(
                    key="FX.*.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Rate",
                    field="mid",
                    quote_interval="100D.0D",
                ),
            ],
            suppliers=models.MarketContextSuppliers(
                commodity="Client",
                credit="Client",
                equity="Client",
                fx="Client",
                rates="Client",
            ),
            options=models.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="ClientInternal",
                default_scope=scope,
                attempt_to_infer_missing_fx=True,
            ),
        ),
        pricing=models.PricingContext(
            model_rules=[
                models.VendorModelRule(
                    supplier="Lusid", model_name="SimpleStatic", instrument_type="Bond"
                )
            ]
        ),
    )

    upsert_configuration_recipe_response = recipes_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )

    print(upsert_configuration_recipe_response)


def create_txn_types(api_factory):

    scope = "Finbourne-Examples"
    
    system_configuration = api_factory.build(lusid.api.SystemConfigurationApi)

    response = system_configuration.set_transaction_configuration_source(
        source="abor",
        set_transaction_configuration_source_request=[
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="FundsIn",
                        description="Deposit New Funds",
                        transaction_class="CashTransfers",
                        transaction_group="abor",
                        transaction_roles="Longer",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="Subscription",
                        movement_types="CashReceivable",
                        side="Side1",
                        direction=1,
                        movement_options=["Capital"],
                        mappings=[
                            models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/CashType",
                            set_to="CashAtBank"
                        
                        )
                        ]
                    )
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="FundsOut",
                        description="Deposit New Funds",
                        transaction_class="CashTransfers",
                        transaction_group="abor",
                        transaction_roles="Shorter",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="Redemption",
                        movement_types="CashReceivable",
                        side="Side1",
                        direction=-1,
                        movement_options=["Capital"],
                        mappings=[
                            models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/CashType",
                            set_to="CashAtBank"
                        
                        )]),
                
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="CashIn",
                        description="New cash into portfolio",
                        transaction_class="CashTransfers",
                        transaction_group="abor",
                        transaction_roles="Longer",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="CashIn",
                        movement_types="CashReceivable",
                        movement_options=["Capital"],
                        side="Side1",
                        direction=1
                    )
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="Buy",
                        description="Buy",
                        transaction_class="Buy",
                        transaction_group="abor",
                        transaction_roles="AllRoles",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="Bought",
                        movement_types="StockMovement",
                        side="Side1",
                        direction=1,
                        movement_options=[],
                        mappings=[
                            models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/CashType",
                            set_to="NonCashInvestments"
                        
                        )]
                    ),
                    models.TransactionConfigurationMovementDataRequest(
                        name="CashInvested",
                        movement_types="CashCommitment",
                        side="Side2",
                        direction=-1,
                        movement_options=[],
                        mappings=[
                            models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/CashType",
                            set_to="CashAtBank"
                        
                        )]
                        ,
                        
                        
                    ),
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="Sell",
                        description="Sell",
                        transaction_class="Sell",
                        transaction_group="abor",
                        transaction_roles="AllRoles",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="Sale",
                        movement_types="StockMovement",
                        side="Side1",
                        direction=-1,
                        movement_options=[],
                         mappings=[
                            models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/CashType",
                            set_to="NonCashInvestments"
                        
                        )]
                    ),
                    models.TransactionConfigurationMovementDataRequest(
                        name="CashProceeds",
                        movement_types="CashCommitment",
                        side="Side2",
                        direction=1,
                        movement_options=[],
                        mappings=[
                            models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/CashType",
                            set_to="CashAtBank"
                        
                        )]
                    ),
                ],
            ),
            models.SetTransactionConfigurationSourceRequest(
                aliases=[
                    models.TransactionConfigurationTypeAlias(
                        type="FxSpotBuy",
                        description="FxSpotBuy",
                        transaction_class="FxSpotBuy",
                        transaction_group="abor",
                        transaction_roles="AllRoles",
                    )
                ],
                movements=[
                    models.TransactionConfigurationMovementDataRequest(
                        name="FxSpotBuyLeg",
                        movement_types="CashCommitment",
                        side="Side1",
                        direction=1,
                        movement_options=[],
                        mappings=[
                            models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/CashType",
                            set_to="CashAtBank"
                        
                        )]
                    ),
                    models.TransactionConfigurationMovementDataRequest(
                        name="FxSpotSellLeg",
                        movement_types="CashCommitment",
                        side="Side2",
                        direction=-1,
                        movement_options=[],
                        mappings=[
                            models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/CashType",
                            set_to="CashAtBank"
                        
                        )]
                    ),
                ],
            ),
        ],
    )
    
    return response


if __name__ == "__main__":
    token = RefreshingToken()

    if token is not None:
        lusid_api_factory = LusidApiClientFactory(token=token)

    else:
        ap = argparse.ArgumentParser()
        ap.add_argument("-s", "--secrets", type=str, help="full path to json file")
        args = ap.parse_args()
        secrets_file = args.secrets
        lusid_api_factory = LusidApiClientFactory(api_secrets_filename=secrets_file)

    create_recipe(lusid_api_factory)
    create_txn_types(lusid_api_factory)

{'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-oscar.lusid.com/app/insights/logs/0HN29JEG2FM2Q:0000000F',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'value': datetime.datetime(2024, 3, 21, 17, 11, 34, 897967, tzinfo=tzlocal())}


### Create portfolio

In this section we load a Transaction portfolio into LUSID.

In [27]:
portfolio_load_sql = f"""
@@scope = select 'FBN_Funds';
@@portfolioCode = select 'FBN_Fund_1';
@@writeAction = select 'Upsert';
@@description = select 'FBN Fund 1';

@createPortfolio = select 'Transaction' as PortfolioType,
@@scope as PortfolioScope,
@@portfolioCode as PortfolioCode,
@@scope  as InstrumentScopes,
@@description as DisplayName,
@@portfolioCode  as Description,
#2024-01-01# as Created,
'' as SubHoldingKeys,
'GBP' as BaseCurrency,
@@writeAction as WriteAction
;


select *
from Lusid.Portfolio.Writer
where ToWrite = @createPortfolio;
"""
run_luminesce(portfolio_load_sql)

,PortfolioScope,PortfolioCode,PortfolioType,BaseCurrency,DisplayName,Description,ParentPortfolioScope,ParentPortfolioCode,CorporateActionSourceScope,CorporateActionSourceId,...,Created,InstrumentScopes,AmortisationMethod,TransactionTypeScope,CashGainLossCalculationDate,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,FBN_Fund_1,Transaction,GBP,FBN Fund 1,FBN_Fund_1,NaN,NaN,NaN,NaN,...,2024-01-01,FBN_Funds,NoAmortisation,default,Default,Insert,2024-03-21 12:09:25.488,0,NaN,NaN


### Create instrument

In this section we load some equity and bond instruments into LUSID.

In [28]:
equity_instrument_load_sql = f"""
@@scope = select 'FBN_Funds';

@instrumentsData= 
values
('BP', 'FBNABOR001', 'GBP'),
('HSBC PLC', 'FBNABOR002', 'GBP'),
('Vodafone Group', 'FBNABOR003', 'GBP'),
('Microsoft Corp', 'FBNABOR004', 'USD'),
('Lloyds Banking Group', 'FBNABOR005', 'GBP');

@instrumentsForUpload = select
column1 as DisplayName,
column2 as ClientInternal,
column3 as DomCcy,
@@scope as Scope
from @instrumentsData;

-- Step 2: Upload the transformed data into LUSID

select *
from Lusid.Instrument.Equity.Writer
where ToWrite = @instrumentsForUpload;
"""
run_luminesce(equity_instrument_load_sql)


,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail,WriteAction,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,...,AsAtCreated,UserIdCreated,RequestIdCreated,AsAtModified,UserIdModified,RequestIdModified,AsAtVersionNumber,Identifiers,DomCcy,LotSize
0,2024-03-21 12:26:17.603,0,NaN,NaN,Upsert,LUID_00003DBN,NaN,NaN,NaN,NaN,...,2024-03-21 12:26:17.603,00urv1yl4m7stbMC42p7,e26481cf-fd3e-4825-83f6-85c103a8feeb,2024-03-21 12:26:17.603,00urv1yl4m7stbMC42p7,e26481cf-fd3e-4825-83f6-85c103a8feeb,1,NaN,USD,1
1,2024-03-21 12:26:17.603,0,NaN,NaN,Upsert,LUID_00003DBO,NaN,NaN,NaN,NaN,...,2024-03-21 12:26:17.603,00urv1yl4m7stbMC42p7,e26481cf-fd3e-4825-83f6-85c103a8feeb,2024-03-21 12:26:17.603,00urv1yl4m7stbMC42p7,e26481cf-fd3e-4825-83f6-85c103a8feeb,1,NaN,GBP,1
2,2024-03-21 12:26:17.603,0,NaN,NaN,Upsert,LUID_00003DBP,NaN,NaN,NaN,NaN,...,2024-03-21 12:26:17.603,00urv1yl4m7stbMC42p7,e26481cf-fd3e-4825-83f6-85c103a8feeb,2024-03-21 12:26:17.603,00urv1yl4m7stbMC42p7,e26481cf-fd3e-4825-83f6-85c103a8feeb,1,NaN,GBP,1
3,2024-03-21 12:26:17.603,0,NaN,NaN,Upsert,LUID_00003DBQ,NaN,NaN,NaN,NaN,...,2024-03-21 12:26:17.603,00urv1yl4m7stbMC42p7,e26481cf-fd3e-4825-83f6-85c103a8feeb,2024-03-21 12:26:17.603,00urv1yl4m7stbMC42p7,e26481cf-fd3e-4825-83f6-85c103a8feeb,1,NaN,GBP,1
4,2024-03-21 12:26:17.603,0,NaN,NaN,Upsert,LUID_00003DBR,NaN,NaN,NaN,NaN,...,2024-03-21 12:26:17.603,00urv1yl4m7stbMC42p7,e26481cf-fd3e-4825-83f6-85c103a8feeb,2024-03-21 12:26:17.603,00urv1yl4m7stbMC42p7,e26481cf-fd3e-4825-83f6-85c103a8feeb,1,NaN,GBP,1


In [7]:
bond_instrument_load_sql = f"""
@@scope = select 'FBN_Funds';

-- Step 1: Define the bond instruments

@bondsData= 
values
('WELLTR 2.517 02/07/2118', 'FBNBND001',  '2023-01-01',  2.517, 'GBP', '1Y', 'ActAct', '2118-07-02'),
('CAMBRG 2.35 06/27/2078', 'FBNBND002',  '2023-01-01',  2.35, 'GBP', '1Y', 'ActAct', '2078-01-01'),
('HTHROW 4 3/8 03/01/27', 'FBNBND003',  '2023-01-01',  4, 'GBP', '1Y', 'ActAct', '2027-03-01'),
('RABOBK 5 3/8 08/03/60', 'FBNBND004',  '2023-01-01',  5, 'GBP', '1Y', 'ActAct', '2060-08-03'),
('COOPWH 6 1/4 07/08/26', 'FBNBND005',  '2023-01-01',  6, 'GBP', '1Y', 'ActAct', '2026-07-08'),
;


@bondsDataForUpload = 
select
Column1 as DisplayName,
column2 as ClientInternal,
column3 as StartDate,
column4 as CouponRate,
column5 as DomCcy,
column5 as FlowConventionsCurrency,
column6 as FlowConventionsPaymentFrequency,
column7 as FlowConventionsDayCountConvention,
'MF' as FlowConventionsRollConvention,
column5 as FlowConventionsPaymentCalendars,
column5 as FlowConventionsResetCalendars,
0 as FlowConventionsSettleDays,
0 as FlowConventionsResetDays,
1 as Principal,
column8 as MaturityDate,
@@scope as Scope
from @bondsData;

-- Step 2: Upload the transformed data into LUSID

select *
from Lusid.Instrument.Bond.Writer
where ToWrite = @bondsDataForUpload;
"""
run_luminesce(bond_instrument_load_sql)

,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail,WriteAction,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,...,CalculationType,RoundingConventions1FaceValue,RoundingConventions1Precision,RoundingConventions1RoundingTarget,RoundingConventions1RoundingType,RoundingConventions2FaceValue,RoundingConventions2Precision,RoundingConventions2RoundingTarget,RoundingConventions2RoundingType,OriginalIssuePrice
0,2024-02-27 10:46:59.395,0,NaN,NaN,Upsert,LUID_00003DAK,NaN,NaN,NaN,NaN,...,Standard,0,0,NaN,NaN,0,0,NaN,NaN,NaN
1,2024-02-27 10:46:59.395,0,NaN,NaN,Upsert,LUID_00003DAM,NaN,NaN,NaN,NaN,...,Standard,0,0,NaN,NaN,0,0,NaN,NaN,NaN
2,2024-02-27 10:46:59.395,0,NaN,NaN,Upsert,LUID_00003DAL,NaN,NaN,NaN,NaN,...,Standard,0,0,NaN,NaN,0,0,NaN,NaN,NaN
3,2024-02-27 10:46:59.395,0,NaN,NaN,Upsert,LUID_00003DAJ,NaN,NaN,NaN,NaN,...,Standard,0,0,NaN,NaN,0,0,NaN,NaN,NaN


### Transactions

In this section we create some transactions in this portfolio for the instruments we just created.

In [30]:
transaction_sql = f"""

@@scope = select 'FBN_Funds';
@@portfolioCode = select 'FBN_Fund_1';

-- Step 1: Define some transactions

@transactions = 
values

-- Equity Transactions
(@@scope, @@portfolioCode, 'txn_001', 'Buy', '2024-01-12', '2024-01-12', 10000, 161.02, 1610200, 'GBP', 'FBNABOR001', 1),
(@@scope, @@portfolioCode, 'txn_003', 'Buy', '2024-01-12', '2024-01-12', 10000, 16.582, 65820,'GBP', 'FBNABOR002', 1),
(@@scope, @@portfolioCode, 'txn_004', 'Buy', '2024-01-12', '2024-01-12', 10000, 77.35, 773500, 'GBP', 'FBNABOR003', 1),
(@@scope, @@portfolioCode, 'txn_011', 'Buy', '2024-01-12', '2024-01-12', 10000, 330, 3300000, 'USD', 'FBNABOR004', 1),
(@@scope, @@portfolioCode, 'txn_002', 'Buy', '2024-01-12', '2024-01-12', 1000, 46,46000, 'GBP', 'FBNABOR005', 1),

--Bond transactions
(@@scope, @@portfolioCode, 'txn_006', 'Buy', '2024-01-12', '2024-01-12', 10, 55, 55682.6, 'GBP', 'FBNBND001', 1),
(@@scope, @@portfolioCode, 'txn_005', 'Buy', '2024-01-12', '2024-01-12', 10, 66, 66000, 'GBP', 'FBNBND002', 1),
(@@scope, @@portfolioCode, 'txn_009', 'Buy', '2024-01-12', '2024-01-12', 10, 90, 90767.12, 'GBP', 'FBNBND003', 1),
(@@scope, @@portfolioCode, 'txn_007', 'Buy', '2024-01-12', '2024-01-12', 10, 90, 90986.64, 'GBP', 'FBNBND004', 1),
(@@scope, @@portfolioCode, 'txn_008', 'Buy', '2024-01-12', '2024-01-12', 10, 98, 98592.47, 'GBP', 'FBNBND005', 1)

;


-- Step 2: Load transactions into LUSID

@createTransactions = 
select
column1 as PortfolioScope,
column2 as PortfolioCode,
column3 as TxnId,
column4 as Type,
column5 as TransactionDate,
column6 as SettlementDate,
column7 as Units,
column8 as TradePrice,
column9 as TotalConsideration,
column10 as SettlementCurrency,
column11 as ClientInternal,
column12 as TradeToPortfolioRate,
'abor' as Source
from @transactions;

-- Upload the transformed data into LUSID

select *
from Lusid.Portfolio.Txn.Writer
where ToWrite = @createTransactions;
"""
run_luminesce(transaction_sql)

,PortfolioScope,PortfolioCode,DisplayName,TxnId,Type,TransactionDate,SettlementDate,Units,TradePrice,TradePriceType,...,ShareClassFigi,Wertpapier,RIC,QuotePermId,EdiKey,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,FBN_Fund_1,FBN Fund 1,txn_001,Buy,2024-01-12,2024-01-12,10000,161.020,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-03-21 14:42:29.660,0,NaN,NaN
1,FBN_Funds,FBN_Fund_1,FBN Fund 1,txn_003,Buy,2024-01-12,2024-01-12,10000,16.582,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-03-21 14:42:29.660,0,NaN,NaN
2,FBN_Funds,FBN_Fund_1,FBN Fund 1,txn_004,Buy,2024-01-12,2024-01-12,10000,77.350,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-03-21 14:42:29.660,0,NaN,NaN
3,FBN_Funds,FBN_Fund_1,FBN Fund 1,txn_011,Buy,2024-01-12,2024-01-12,10000,330.000,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-03-21 14:42:29.660,0,NaN,NaN
4,FBN_Funds,FBN_Fund_1,FBN Fund 1,txn_002,Buy,2024-01-12,2024-01-12,1000,46.000,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-03-21 14:42:29.660,0,NaN,NaN
5,FBN_Funds,FBN_Fund_1,FBN Fund 1,txn_006,Buy,2024-01-12,2024-01-12,10,55.000,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-03-21 14:42:29.660,0,NaN,NaN
6,FBN_Funds,FBN_Fund_1,FBN Fund 1,txn_005,Buy,2024-01-12,2024-01-12,10,66.000,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-03-21 14:42:29.660,0,NaN,NaN
7,FBN_Funds,FBN_Fund_1,FBN Fund 1,txn_009,Buy,2024-01-12,2024-01-12,10,90.000,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-03-21 14:42:29.660,0,NaN,NaN
8,FBN_Funds,FBN_Fund_1,FBN Fund 1,txn_007,Buy,2024-01-12,2024-01-12,10,90.000,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-03-21 14:42:29.660,0,NaN,NaN
9,FBN_Funds,FBN_Fund_1,FBN Fund 1,txn_008,Buy,2024-01-12,2024-01-12,10,98.000,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-03-21 14:42:29.660,0,NaN,NaN


### Quotes

In this section we insert some quotes for the instruments we just traded and also some FX rates for currency conversions in case the transactions are not in the currency of the portfolio.

In [126]:
quote_sql = f"""

-- Load quotes from an Excel worksheet

@quotes_data = use Drive.Excel
--file=/abor-demo-data/prices/aborquotes.xlsx
--worksheet=prices
enduse;

-- Transform quote data

@quotes_for_upload = select
'ClientInternal' as InstrumentIdType,
client_internal as Instrumentid,
'FBN_Funds' as QuoteScope,
'Price' as QuoteType,
'Lusid' as Provider,
'Mid' as Field,
price_date as QuoteEffectiveAt,
close_price as Value,
units as Unit
from @quotes_data;

-- Upload quotes into LUSID

select * from Lusid.Instrument.Quote.Writer
where ToWrite = @quotes_for_upload;
"""
run_luminesce(quote_sql)

,QuoteScope,Provider,PriceSource,InstrumentId,InstrumentIdType,QuoteType,Field,Lineage,ScaleFactor,QuoteEffectiveAt,Value,Unit,CutLabel,UploadedBy,QuoteAsAt,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,Lusid,NaN,NaN,ClientInternal,Price,Mid,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,Upsert,NaN,151,InstrumentId,A non-empty value is required
1,FBN_Funds,Lusid,NaN,FBNABOR001,ClientInternal,Price,Mid,NaN,NaN,2024-02-17,4.765000,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 17:25:21.862,Upsert,2024-03-21 17:25:21.862,0,NaN,NaN
2,FBN_Funds,Lusid,NaN,FBNBND005,ClientInternal,Price,Mid,NaN,NaN,2024-02-29,80.599998,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 17:25:21.862,Upsert,2024-03-21 17:25:21.862,0,NaN,NaN
3,FBN_Funds,Lusid,NaN,FBNABOR001,ClientInternal,Price,Mid,NaN,NaN,2024-03-09,4.900000,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 17:25:21.862,Upsert,2024-03-21 17:25:21.862,0,NaN,NaN
4,FBN_Funds,Lusid,NaN,FBNBND004,ClientInternal,Price,Mid,NaN,NaN,2024-02-21,93.589996,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 17:25:21.862,Upsert,2024-03-21 17:25:21.862,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,FBN_Funds,Lusid,NaN,FBNBND004,ClientInternal,Price,Mid,NaN,NaN,2024-01-14,98.879997,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 17:25:21.862,Upsert,2024-03-21 17:25:21.862,0,NaN,NaN
559,FBN_Funds,Lusid,NaN,FBNBND004,ClientInternal,Price,Mid,NaN,NaN,2024-03-02,95.680000,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 17:25:21.862,Upsert,2024-03-21 17:25:21.862,0,NaN,NaN
560,FBN_Funds,Lusid,NaN,FBNBND003,ClientInternal,Price,Mid,NaN,NaN,2024-01-16,93.529999,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 17:25:21.862,Upsert,2024-03-21 17:25:21.862,0,NaN,NaN
561,FBN_Funds,Lusid,NaN,FBNBND002,ClientInternal,Price,Mid,NaN,NaN,2024-03-11,94.029999,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 17:25:21.862,Upsert,2024-03-21 17:25:21.862,0,NaN,NaN


In [103]:
fxrate_sql = f"""
@fx_price_data =
use Drive.Excel
--file=/abor-demo-data/prices/daily_quotes.xlsx
--worksheet=fx
enduse;

-- Run transformation on the data

@quotes_for_upload = select
'CurrencyPair' as InstrumentIdType,
ccy_pair as Instrumentid,
'FBN_Funds' as QuoteScope,
'Rate' as QuoteType,
'Lusid' as Provider,
'Mid' as Field,
price_date as QuoteEffectiveAt,
'12' as Value,
'GBP' as Unit
from @fx_price_data;

-- Upload quotes into LUSID

select * from Lusid.Instrument.Quote.Writer
where ToWrite = @quotes_for_upload;
"""
run_luminesce(fxrate_sql)

,QuoteScope,Provider,PriceSource,InstrumentId,InstrumentIdType,QuoteType,Field,Lineage,ScaleFactor,QuoteEffectiveAt,Value,Unit,CutLabel,UploadedBy,QuoteAsAt,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Rate,Mid,NaN,NaN,2024-01-26,12,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 16:59:36.398,Upsert,2024-03-21 16:59:36.398,0,NaN,NaN
1,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Rate,Mid,NaN,NaN,2024-02-15,12,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 16:59:36.398,Upsert,2024-03-21 16:59:36.398,0,NaN,NaN
2,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Rate,Mid,NaN,NaN,2024-02-02,12,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 16:59:36.398,Upsert,2024-03-21 16:59:36.398,0,NaN,NaN
3,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Rate,Mid,NaN,NaN,2024-02-13,12,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 16:59:36.398,Upsert,2024-03-21 16:59:36.398,0,NaN,NaN
4,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Rate,Mid,NaN,NaN,2024-02-22,12,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 16:59:36.398,Upsert,2024-03-21 16:59:36.398,0,NaN,NaN
5,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Rate,Mid,NaN,NaN,2024-02-07,12,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 16:59:36.398,Upsert,2024-03-21 16:59:36.398,0,NaN,NaN
6,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Rate,Mid,NaN,NaN,2024-02-16,12,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 16:59:36.398,Upsert,2024-03-21 16:59:36.398,0,NaN,NaN
7,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Rate,Mid,NaN,NaN,2024-01-17,12,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 16:59:36.398,Upsert,2024-03-21 16:59:36.398,0,NaN,NaN
8,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Rate,Mid,NaN,NaN,2024-01-30,12,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 16:59:36.398,Upsert,2024-03-21 16:59:36.398,0,NaN,NaN
9,FBN_Funds,Lusid,NaN,GBP/USD,CurrencyPair,Rate,Mid,NaN,NaN,2024-02-26,12,GBP,NaN,00urv1yl4m7stbMC42p7,2024-03-21 16:59:36.398,Upsert,2024-03-21 16:59:36.398,0,NaN,NaN


### Cash transactions

In this section we create some cash transactions

In [69]:
cash_transaction_sql = f"""
@@scope = select 'FBN_Funds';
@@portfolioCode = select 'FBN_Fund_1';



/*  
    Step 1: Define some cash transactions
    
    In the step below, we create a transaction type called AborFundsIn which is configured to create
    Capital movements in the Journal Entry

*/

@transactions = 
values
(@@scope, @@portfolioCode, 'txn-0010', 'FundsIn', '2024-01-12', '2024-01-12', 20000000, 1, 20000000, 'GBP', 'CCY_GBP', 1, 1),

--Fx Spots
(@@scope, @@portfolioCode, 'TXN-25', 'FxSpotBuy', '2024-01-12', '2024-01-12', 3300000, 1.19, 2773109.24, 'USD', 'CCY_USD', 1, 1)
;




@createTransactions = 
select
column1 as PortfolioScope,
column2 as PortfolioCode,
column3 as TxnId,
column4 as Type,
column5 as TransactionDate,
column6 as SettlementDate,
column7 as Units,
column8 as TradePrice,
column9 as TotalConsideration,
column10 as SettlementCurrency,
column11 as LusidInstrumentId,
Column12 as TradeToPortfolioRate,
column13 as ExchangeRate,
'abor' as Source
from @transactions;

-- Step 2: Load transactions into LUSID

select *
from Lusid.Portfolio.Txn.Writer
where ToWrite = @createTransactions;
"""
run_luminesce(cash_transaction_sql)

,PortfolioScope,PortfolioCode,DisplayName,TxnId,Type,TransactionDate,SettlementDate,Units,TradePrice,TradePriceType,...,ShareClassFigi,Wertpapier,RIC,QuotePermId,EdiKey,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,FBN_Fund_1,FBN Fund 1,txn-0010,FundsIn,2024-01-12,2024-01-12,20000000,1.00,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-03-21 16:15:15.033,0,NaN,NaN
1,FBN_Funds,FBN_Fund_1,FBN Fund 1,TXN-25,FxSpotBuy,2024-01-12,2024-01-12,3300000,1.19,Price,...,NaN,NaN,NaN,NaN,NaN,Upsert,2024-03-21 16:15:15.033,0,NaN,NaN


### Chart of accounts
A chart of accounts (COA) is an index of all financial accounts in the general ledger of a company. We must create a COA before i can add general ledger accounts to it. 


In [70]:
chart_of_accounts_sql = f"""

-- Step 1: Define the Chart of Accounts

@@scope = select 'FBN_Funds';
@@code = select 'FBN_COA_1';
@@name = select 'FBN COA 1';
@@writeAction = select 'Upsert';

@chartOfAccounts =
select
@@scope as ChartOfAccountsScope,
@@code as ChartOfAccountsCode,
@@name as DisplayName,
@@name as Description,
@@writeAction as WriteAction;

-- Step 2: Upload Chart of Account into LUSID

select * from Lusid.ChartOfAccounts.Writer where ToWrite = @chartOfAccounts;
"""
run_luminesce(chart_of_accounts_sql)

,ChartOfAccountsScope,ChartOfAccountsCode,DisplayName,Description,WriteAction,WriteAsAt,WriteErrorCode,WriteError,WriteErrorDetail
0,FBN_Funds,FBN_COA_1,FBN COA 1,FBN COA 1,Insert,2024-03-21 16:16:15.721,0,NaN,NaN
